In [1]:
!pip install bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.4 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from peft import PeftModel, PeftConfig, PeftModelForCausalLM
from transformers import  BitsAndBytesConfig
import torch
import os

from tqdm import tqdm
import argparse

def retain_content_before_last_non_english_punctuation(text):
    # Reverse iterate over the text from the end
    for i in range(len(text) - 1, 0, -1):
        # Check if the current character is a space and the previous one is an English letter
        if text[i] == ' ' and text[i-1].isalpha():
            # Return the text up to the space
            return text[:i+2]
    # Return the original text if no such space is found
    return text

def generate(prompt, model, tokenizer):
    generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 64,
    "num_return_sequences": 1,}
    prompt = f"<s> GEC: {prompt}\nCorrected:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        output = model.generate(**inputs, **generation_kwargs)
    output = output[0].to("cpu")
    printed = tokenizer.decode(output, skip_special_tokens=True)
    printed = (printed.split('\n')[1])
    printed = retain_content_before_last_non_english_punctuation(printed)
    return printed.replace("Corrected: ", "")





model_path = "jiajunzhu/comp550"


tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16,quantization_config=BitsAndBytesConfig(load_in_4bit=True), load_in_4bit=True, device_map="cuda:0")
model.eval()
corrupted_text = "I like to swimming"
correct_text = generate(corrupted_text, model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [3]:
correct_text

'I like swimming .'